In [ ]:
import torch
from psutil import virtual_memory
torch.cuda.is_available()
GPUmem = torch.cuda.mem_get_info()
mem = GPUmem[1] /1024 / 1024 / 1024
mem += virtual_memory().total /1024 / 1024 / 1024 / 2
mem

27.873252868652344

In [4]:
import WebDisplay
data = {
    'name': 'example',
    'value': 42,
    'status': 'active'
}
web_display = WebDisplay.WebDisplay(data)
web_display.print_data()

name: example
value: 42
status: active


In [4]:
from bs4 import BeautifulSoup
from pytubefix import YouTube
yt = YouTube('https://music.youtube.com/watch?v=NycFr6D6DSw&si=przy_9oxIVQs5HI9')
print('download...')
yt.streams.get_audio_only().download(output_path='download/')   #下載音訊
print('ok!')

download...
ok!


In [5]:
acc = yt.streams.get_audio_only()
acc.download(output_path='download/')   #下載音訊

'f:\\NFU\\score\\download\\春日影 (MyGO!!!!! ver.).m4a'

In [3]:
from YouTubeDownload import YouTubeDownload
from MusicSeparation import MusicSeparation
import os
url = 'https://youtu.be/sHD_z90ZKV0?list=RDsHD_z90ZKV0'
vedio = YouTubeDownload.download_youtube_video(url)
audio = YouTubeDownload.download_youtube_audio(url)
separation_audio = MusicSeparation.run_separation(audio)
merge = YouTubeDownload.merge_video_audio(vedio, separation_audio)
print(vedio, audio , merge, separation_audio)
#os.system(merge)
separation_audio

已儲存：f:\NFU\score\score/output\周杰倫 Jay Chou【稻香 Rice Field】-Official Music Video\周杰倫 Jay Chou【稻香 Rice Field】-Official Music Video-vocals.wav
已儲存：f:\NFU\score\score/output\周杰倫 Jay Chou【稻香 Rice Field】-Official Music Video\周杰倫 Jay Chou【稻香 Rice Field】-Official Music Video-other.wav
"f:\NFU\score\download\周杰倫 Jay Chou【稻香 Rice Field】-Official Music Video.mp4" "f:\NFU\score\download\周杰倫 Jay Chou【稻香 Rice Field】-Official Music Video.m4a" "f:\NFU\score\download\周杰倫 Jay Chou【稻香 Rice Field】-Official Music Video_merged.mp4" f:\NFU\score\score/output\周杰倫 Jay Chou【稻香 Rice Field】-Official Music Video\周杰倫 Jay Chou【稻香 Rice Field】-Official Music Video-other.wav


'f:\\NFU\\score\\score/output\\周杰倫 Jay Chou【稻香 Rice Field】-Official Music Video\\周杰倫 Jay Chou【稻香 Rice Field】-Official Music Video-other.wav'

In [2]:
os.system(merge)
audio

'"f:\\NFU\\score\\download\\千本桜MV\u30002025年版 White Flame feat. 初音ミク.m4a"'